<a href="https://colab.research.google.com/github/akashrama/capstone/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install cftime

     |████████████████████████████████| 317kB 2.8MB/s 


In [0]:
import pandas as pd
import pandas.tseries
import numpy as np

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

import xarray as xr

%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
def process(path):
  opened = xr.open_dataset(path)
  df = opened.sel(lat=slice(-56,12), lon=slice(277,325.5), bnds=0).groupby('time.year').mean().to_dataframe()
  df = df.reset_index()
  df = df.drop(['lat','lon'], axis=1)
  return df

In [0]:
pr = process('/content/drive/My Drive/Colab Notebooks/capstone/Noaa/pr_A2.20010101-21001231.nc')
tas = process('/content/drive/My Drive/Colab Notebooks/capstone/Noaa/tas_A1.200101-210012-2.nc')
tasmax = process('/content/drive/My Drive/Colab Notebooks/capstone/Noaa/tasmax_A2.20010101-21001231-2.nc')
tasmin = process('/content/drive/My Drive/Colab Notebooks/capstone/Noaa/tasmin_A2.20010101-21001231.nc')

In [0]:
def noaa(pr, tas, tasmax, tasmin):
  lat = pr['lat_bnds']
  lon = pr['lon_bnds']
  year = pr['year']
  pr = pr['pr'] * 60 * 60 * 24 * 10
  tas = tas['tas']
  tasmax = tasmax['tasmax']
  tasmin = tasmin['tasmin']
  exp = pd.concat([lat, lon, pr, tas, tasmax, tasmin, year], axis=1)
  exp = exp.groupby(['year', 'lat_bnds', 'lon_bnds']).mean()
  exp = exp.reset_index()
  return exp

In [0]:
exp = noaa(pr, tas, tasmax, tasmin)

In [0]:
def read_csv():
    
    ''' This function imports necessary data'''
    
    df_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/capstone/dengue_data/submission_format.csv')
    df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/capstone/dengue_data/dengue_features_test.csv')
    df_training_labels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/capstone/dengue_data/dengue_labels_train.csv')
    df_training_features = pd.read_csv('/content/drive/My Drive/Colab Notebooks/capstone/dengue_data/dengue_features_train.csv')
    return (df_submission, df_test, df_training_labels, df_training_features)


In [0]:
df_submission, df_test, df_training_labels, df_training_features = read_csv()

In [11]:
errors = abs(predict - y_testtime)

NameError: ignored

In [0]:
print('Metrics for Random Forest Trained on Original Data')
MAE = round(np.mean(errors), 2)
print('Mean absolute error:', MAE, 'degrees.')
mape = 100 * (MAE / round(np.mean(y_testtime), 2))
print('Mean absolute percentage:', mape)

In [0]:
def forecast(exp, df_test, df_training_labels, df_training_features):
  X_train = df_training_features
  y_train = df_training_labels
  X_test = df_test

  y_train = y_train['total_cases']

  X_train = pd.concat([X_train, y_train], axis=1)

  X_train = X_train[['reanalysis_sat_precip_amt_mm', 'reanalysis_avg_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 'year', 'total_cases']]
  X_test = X_test[['reanalysis_sat_precip_amt_mm', 'reanalysis_avg_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 'year']]

  X_train = X_train.dropna()
  X_train = X_train.reset_index(drop=True)
  X_test = X_test.dropna()
  X_test = X_test.reset_index(drop=True)

  y_train = X_train['total_cases']
  y_train = y_train.reset_index()

  X_train = X_train.drop('total_cases', axis=1)
  y_train = y_train.drop('index', axis=1)

  X_train = X_train.drop(['year'], axis=1)
  X_test = X_test.drop(['year'], axis=1)

  exp_lat = exp['lat_bnds']
  exp_lon = exp['lon_bnds']
  exp_year = exp['year']

  exp = exp.drop(['lat_bnds','lon_bnds', 'year'], axis=1)

  X_train.columns = ['pr', 'tas', 'tasmax', 'tasmin']

  rf = RandomForestRegressor(
    bootstrap= True,
    max_depth= 1000,
    max_features= 'sqrt',
    min_samples_leaf= 50,
    min_samples_split= 30,
    n_estimators= 800)
  rf.fit(X_train, y_train)
  future_pred = rf.predict(exp)

  result = {'total_cases': future_pred}
  
  future_df = pd.DataFrame(result)

  final = pd.concat([exp, exp_year, exp_lat, exp_lon, future_df], axis=1)
  final_g = final.groupby(['lat_bnds', 'lon_bnds', 'year']).sum()
  return final_g

In [13]:
final = forecast(exp, df_test, df_training_labels, df_training_features)
final

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


pr         tas  ...      tasmin  total_cases
lat_bnds   lon_bnds year                         ...                         
-56.629215 277.5    2001  26.823822  279.609741  ...  278.989227    15.801354
                    2002  29.136833  279.609711  ...  278.950592    15.802572
                    2003  33.108948  279.905640  ...  279.256989    15.982121
                    2004  32.287910  279.651794  ...  279.051941    15.941381
                    2005  28.135958  279.369843  ...  278.760437    15.802572
...                             ...         ...  ...         ...          ...
 10.112360 322.5    2096  50.298172  299.497955  ...  299.242828    38.095417
                    2097  52.781776  298.925507  ...  298.663513    38.861730
                    2098  29.248409  299.037476  ...  298.782135    38.703014
                    2099  44.304260  299.054199  ...  298.785309    38.441845
                    2100  49.479324  299.175323  ...  298.921478    39.097338

[64600 rows x 5 columns]

In [0]:
export_csv = final.to_csv(r'/content/drive/My Drive/Colab Notebooks/final/not.csv', index = None, header=True) 